# K2-24

RadVel and exoplanet both have excellent tutorials where they fit for the two planets b and c of the K2-24 system. We will do the same here to show how to fit RV data in Ravest.

**Links:**  
K2-24 paper (Petigura et al 2016 ApJ 818 36): https://doi.org/10.3847/0004-637X/818/1/36  
RadVel tutorial: https://radvel.readthedocs.io/en/latest/tutorials/K2-24_Fitting+MCMC.html  
exoplanet tutorial: https://gallery.exoplanet.codes/tutorials/rv/  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import ravest.prior
from ravest.fit import Fitter
from ravest.model import calculate_mpsini
from ravest.param import Parameter, Parameterisation

Import the data

In [ ]:
url = "https://raw.githubusercontent.com/California-Planet-Search/radvel/master/example_data/epic203771098.csv"
data = pd.read_csv(url, usecols=[1,2,3], names=["errvel", "time", "vel"], skiprows=1)
data

In [ ]:
plt.figure(figsize=(15,3.5))
plt.title("K2-24 radial velocity data")
plt.ylabel("Radial Velocity [m/s]")
plt.xlabel("BJD_TDB - 2454833")
plt.errorbar(data["time"], data["vel"], yerr=data["errvel"], marker=".", linestyle="None")
plt.show()

Create a `Fitter` object, and choose which parameterisation to fit with, whether to fix or fit for each parameter, and the initial parameter values. We can fit a circular model by fixing eccentricity $e=0$ (the argument of periapsis $\omega_\star$ is now degenerate and can be fixed at any value, by convention we fix at $\pi/2$.) The reference zero-point time `t0` is used for linear and quadratic trends terms $\dot{\gamma}$ and $\ddot{\gamma}$.

In [ ]:
fitter = Fitter(planet_letters=["b","c"], parameterisation=Parameterisation("P K e w Tc"))
fitter.add_data(time=data["time"].to_numpy(), 
                vel=data["vel"].to_numpy(), 
                verr=data["errvel"].to_numpy(), 
                t0=2420)

# Construct the params dict
# These values will be used as your initial guess for the fit
params = {"P_b": Parameter(20.8851, "d", fixed=True),
          "K_b": Parameter(10, "m/s", fixed=False),
          "e_b": Parameter(0, "", fixed=True),
          "w_b": Parameter(np.pi/2, "rad", fixed=True),
          "Tc_b": Parameter(2072.7948, "d", fixed=True),

          "P_c": Parameter(42.3633, "d", fixed=True),
          "K_c": Parameter(10, "m/s", fixed=False),
          "e_c": Parameter(0, "", fixed=True),
          "w_c": Parameter(np.pi/2, "rad", fixed=True),
          "Tc_c": Parameter(2082.6251, "d", fixed=True),
          
          "g": Parameter(0, "m/s", fixed=False),
          "gd": Parameter(0, "m/s/day", fixed=False),
          "gdd": Parameter(0, "m/s/day^2", fixed=True),
          
          "jit": Parameter(0, "m/s", fixed=False),}

fitter.params = params
fitter.params

Define the prior functions for the free parameters. You can see a list of available prior functions at `ravest.prior.PRIOR_FUNCTIONS`.

In [ ]:
ravest.prior.PRIOR_FUNCTIONS

In [ ]:
# Construct the priors dict. Every parameter that isn't fixed requires a prior.
priors = {
          "K_b": ravest.prior.Uniform(0,20),
          "K_c": ravest.prior.Uniform(0,20),

          "g": ravest.prior.Uniform(-10, 10),
          "gd": ravest.prior.Uniform(-1, 1),
          
          "jit": ravest.prior.Uniform(0, 5),
         }

fitter.priors = priors
fitter.priors

Now that we have loaded the `Fitter` with the data, our parameterisation, our initial parameter values, and priors for each of the free parameters, we can now fit the free parameters of the model to the data.  
  
First, Maximum A Posteriori (MAP) optimisation is performed to find the best-fit solution.

In [ ]:
map_results = fitter.find_map_estimate(method="Powell")
map_results

Let's take a look at what the RV looks like with these parameter values:

In [ ]:
fitter.plot_MAP_rv(map_result=map_results)
fitter.plot_MAP_phase(planet_letter="b", map_result=map_results)
fitter.plot_MAP_phase(planet_letter="c", map_result=map_results)

We can use MCMC to more fully explore the parameter space and estimate the parameter uncertainties. For the purposes of making this notebook run quickly, this is only running for 10000 steps - you should run considerably more. `ravest` enforces a minimum of at least 2 walkers per each free parameter, again though you should consider running more. You should also consider using randomly initialised starting points, rather than the MAP solution, to better explore the parameter space. Parallelisation has been (experimentally) enabled via the `multiprocessing` argument, this can result in significant speedups - especially for longer chains/more walkers.

In [ ]:
nwalkers = 4 * len(fitter.free_params_dict)
nsteps = 10000

# Fit the free parameters to the data. Use the MAP solution as the initial value for the MCMC walkers.
fitter.run_mcmc(initial_values=map_results.x, nwalkers=nwalkers, nsteps=nsteps, progress=True, multiprocessing=True)  # This will take a few minutes!

Now that the MCMC is finished, the state of the `emcee` sampler has been saved into the `Fitter` object. We can therefore export the posterior samples, as a `numpy` array that can be passed into other functions (such as for comparing two models by calculating the Bayesian evidence - example notebook coming soon!). We can also export them into a Pandas dataframe, which keeps each parameter labelled. In both cases, we can pass in the `discard_start` and `discard_end` arguments, to drop the burn-in or "zoom" in on certain areas of the chain. The `thin` argument keeps only every `thin`-th sample, and the `flat` argument combines all of the different walkers for each parameter into one long chain per parameter.

In [ ]:
# Get the samples as a numpy array
samples = fitter.get_samples_np(discard_start=1000, discard_end=0, thin=1, flat=False) # shape (nsteps, nwalkers, ndim)

# Get the samples as a labelled Pandas dataframe
samples_df = fitter.get_samples_df(discard_start=1000, discard_end=0, thin=1)  # shape (nsteps*nwalkers, ndim)
samples_df

To inspect the chains visually, we can plot (and optionally save) the time series of each parameter in the chain. We see that after a brief period aorund the initial values (which we expect, as we started the walkers at the MAP values, so they'll be starting at an area of maximised likelihood), they eventually break out and start exploring the parameter space more. For all future plots we can discard this initial "burn-in" with the `discard_start` argument.

In [ ]:
fitter.plot_chains(discard_start=0, discard_end=0, thin=1, save=False)

We can visualise the posterior parameter distributions in corner plots, using the `corner` module.

In [ ]:
fitter.plot_corner(discard_start=1000, discard_end=0, thin=1, save=False)

Inspecting the posteriors, we can see the 16th, 50th and 84th percentiles are shown, which could be used for a quoted value and uncertainty. It's a good idea to inspect the posterior distribution visually with the corner plots though, as they may not always be nice Gaussians, which means those percentiles may not be a good representation. (This is often the case for eccentricity!). For further analysis and inspection, recall that we can get a dataframe of the samples (e.g. to plot them in a histogram to inspect the distribution closer) by using the `Fitter.get_samples_df()` method that we saw earlier.

Let's see what the posterior RV looks like, this time using the median values of each parameter's posterior distribution (i.e. the parameter values quoted in the corner plot above). We'll also look at each planet's contribution in isolation with phase plots.

In [ ]:
fitter.plot_posterior_rv(discard_start=1000, discard_end=0, thin=1)
fitter.plot_posterior_phase("b", discard_start=1000, discard_end=0, thin=1)
fitter.plot_posterior_phase("c", discard_start=1000, discard_end=0, thin=1)

To calculate planetary mass estimates $M_p\sin{i}$, we need to know the stellar mass. Using the value $M_\star=1.12\pm0.05$ used in Petigura et al. 2016, we can generate a distribution of stellar mass values from the published value and uncertainty, and draw from that distribution to use as the value of $M_\star$ in the conversion equation.

In [ ]:
# Stellar mass values from Petigura et al. 2016
mstar_val = 1.12  # [M_sun]
mstar_err = 0.05 # [M_sun]

# Create a distribution of stellar mass values from the published value and uncertainty
np.random.seed(47)  # For reproducibility
mstar = np.random.normal(loc=mstar_val, scale=mstar_err, size=len(samples_df))
# Ensure all values in mstar are positive (incredibly unlikely to be an issue with these values, but just in case)
while any(mstar <= 0):
    mstar[mstar <= 0] = np.random.normal(loc=mstar_val, scale=mstar_err, size=sum(mstar <= 0))

In [ ]:
# get the posterior samples (both free and fixed) as a dictionary
posterior_params = fitter.get_posterior_params_dict(discard_start=1000, discard_end=0, thin=1)  # get the fixed value for fixed parameters, get the MCMC samples for the free parameters

# use the MCMC samples and the stellar mass distribution to get a distribution for Mp sin(i)
mpsini_b = calculate_mpsini(mstar, posterior_params["P_b"], posterior_params["K_b"], posterior_params["e_b"], unit="M_earth")
mpsini_c = calculate_mpsini(mstar, posterior_params["P_c"], posterior_params["K_c"], posterior_params["e_c"], unit="M_earth")

# calculate the median and 1-sigma uncertainties
perc_b = np.percentile(mpsini_b, [16, 50, 84])
perc_c = np.percentile(mpsini_c, [16, 50, 84])
print("Planet b Mpsin(i):", perc_b[1], "+", perc_b[1] - perc_b[0], "-", perc_b[2] - perc_b[1])
print("Planet c Mpsin(i):", perc_c[1], "+", perc_c[1] - perc_c[0], "-", perc_c[2] - perc_c[1])

# Plot the mass posteriors for inspection
plt.hist(mpsini_b, bins=100, histtype="step")
plt.hist(mpsini_c, bins=100, histtype="step")
plt.show()

## Eccentric orbits

Let's make a new `Fitter` object and fit for eccentricity. We'll fit in the $\sqrt{e}\cos{\omega_\star}$ and $\sqrt{e}\sin{\omega_\star}$ parameterisation 

In [ ]:
# Fit in the sqrt(e) parameterisation
parameterisation_se = Parameterisation("P K secosw sesinw Tc")

fitter_se = Fitter(planet_letters=["b","c"], parameterisation=parameterisation_se)
fitter_se.add_data(time=data["time"].to_numpy(), 
                   vel=data["vel"].to_numpy(), 
                   verr=data["errvel"].to_numpy(), 
                   t0=2420)
print(fitter_se.t0)

# Construct the params dict
# These values will be used as your initial guess for the fit
params_se = {"P_b": Parameter(20.8853, "d", fixed=True),
            "K_b": Parameter(np.exp(1.55037), "m/s", fixed=False),
            "secosw_b": Parameter(0.01, "", fixed=False),
            "sesinw_b": Parameter(0.01, "", fixed=False),
            "Tc_b": Parameter(2072.79, "d", fixed=True),

            "P_c": Parameter(42.363, "d", fixed=True),
            "K_c": Parameter(np.exp(1.37648), "m/s", fixed=False),
            "secosw_c": Parameter(0.01, "", fixed=False),
            "sesinw_c": Parameter(0.01, "", fixed=False),
            "Tc_c": Parameter(2082.63, "d", fixed=True),
            
            "g": Parameter(-3.99195, "m/s", fixed=False),
            "gd": Parameter(-0.0292189, "m/s/day", fixed=False),
            "gdd": Parameter(0.00182259, "m/s/day^2", fixed=False),

            "jit": Parameter(2.09753, "m/s", fixed=False),
            }

fitter_se.params = params_se
fitter_se.params

In [ ]:
# Construct the priors dict. Every parameter that isn't fixed requires a prior.
priors_se = {
          "K_b": ravest.prior.Uniform(0,50),
          "secosw_b": ravest.prior.Uniform(-np.sqrt(0.8), np.sqrt(0.8)),
          "sesinw_b": ravest.prior.Uniform(-np.sqrt(0.8), np.sqrt(0.8)),

          "K_c": ravest.prior.Uniform(0,50),
          "secosw_c": ravest.prior.Uniform(-np.sqrt(0.8), np.sqrt(0.8)),
          "sesinw_c": ravest.prior.Uniform(-np.sqrt(0.8), np.sqrt(0.8)),

          "g": ravest.prior.Uniform(-10, 10),
          "gd": ravest.prior.Uniform(-0.1, 0.1),
          "gdd": ravest.prior.Uniform(-0.1, 0.1),
          "jit": ravest.prior.Uniform(0, 5),
        }

fitter_se.priors = priors_se
fitter_se.priors

In [ ]:
map_results_se = fitter_se.find_map_estimate(method="Powell")
map_results_se

In [ ]:
fitter_se.plot_MAP_rv(map_result=map_results_se)
fitter_se.plot_MAP_phase(planet_letter="b", map_result=map_results_se)
fitter_se.plot_MAP_phase(planet_letter="c", map_result=map_results_se)

We can already see that the residuals are generally tighter than they were in the circular case.

In [ ]:
nwalkers = 4 * len(fitter_se.free_params_dict)
nsteps = 10000

# Fit the free parameters to the data
samples_se = fitter_se.run_mcmc(initial_values=map_results_se.x, nwalkers=nwalkers, nsteps=nsteps, progress=True, multiprocessing=True)  # This will take a while!

In [ ]:
# Get the samples as a numpy array
samples_se = fitter_se.get_samples_np(discard_start=1000, discard_end=0, thin=1, flat=False) # shape (nsteps, nwalkers, ndim)

# Get the samples as a labelled Pandas dataframe
samples_df_se = fitter_se.get_samples_df(discard_start=1000, discard_end=0, thin=1)  # shape (nsteps*nwalkers, ndim)
samples_df_se

In [ ]:
fitter_se.plot_chains(discard_start=1000, discard_end=0, thin=1, save=False)

In [ ]:
fitter_se.plot_corner(discard_start=1000, discard_end=0, thin=1, save=False)

We can convert $\sqrt{e}\cos{\omega_\star}$ and $\sqrt{e}\sin{\omega_\star}$ back to $e$ to inspect the posterior eccentricity directly.

In [ ]:
eb_array, wb_array = parameterisation_se.convert_secosw_sesinw_to_e_w(samples_df_se["secosw_b"], samples_df_se["sesinw_b"])
ec_array, wc_array = parameterisation_se.convert_secosw_sesinw_to_e_w(samples_df_se["secosw_c"], samples_df_se["sesinw_c"])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4))
ax1.hist(eb_array, bins=100, histtype="step", label="Planet b")
ax1.hist(ec_array, bins=100, histtype="step", label="Planet c")

ax1.legend()
ax1.set_xlabel("Eccentricity")
ax1.set_ylabel("Number of samples")

ax2.hist(wb_array, bins=100, histtype="step", label="Planet b")
ax2.hist(wc_array, bins=100, histtype="step", label="Planet c")
ax2.legend()
ax2.set_xlabel("Argument of periastron [rad]")
ax2.set_ylabel("Number of samples")
plt.show()

Planet b definitely looks eccentric, harder to tell for planet c though. (Remember $\omega_\star$ becomes undefined when $e=0$, which might explain why $\omega_{\star,c}$ has such a wide spread, as a lot of the samples for $e_c$ are close to 0.)

Let's inspect how well our best-fit parameter values matches the observed RV data.

In [ ]:
fitter_se.plot_posterior_rv(discard_start=1000, discard_end=0, thin=1)
fitter_se.plot_posterior_phase("b", discard_start=1000, discard_end=0, thin=1)
fitter_se.plot_posterior_phase("c", discard_start=1000, discard_end=0, thin=1)

Let's see how allowing for eccentric orbits affects the mass estimates $M_p\sin{i}$. We will use the same stellar mass values from Petigura et al. 2016 that we used earlier.

In [ ]:
# Create a distribution of stellar mass values using the same published value and uncertainty as before
np.random.seed(47)  # For reproducibility
mstar = np.random.normal(loc=mstar_val, scale=mstar_err, size=len(samples_df_se))
# Ensure all values in mstar are positive
while any(mstar <= 0):
    mstar[mstar <= 0] = np.random.normal(loc=mstar_val, scale=mstar_err, size=sum(mstar <= 0))

In [ ]:
# get the posterior samples (both free and fixed) as a dictionary
posterior_params_se = fitter_se.get_posterior_params_dict(discard_start=1000, discard_end=0, thin=1)  # get the fixed value for fixed parameters, get the MCMC samples for the free parameters

# we need to convert secosw and sesinw to e and w
# let's do this using the parameterisation class, parameterisation_se
posterior_params_se["e_b"], posterior_params_se["w_b"] = parameterisation_se.convert_secosw_sesinw_to_e_w(posterior_params_se["secosw_b"], posterior_params_se["sesinw_b"])
# and the same for _c
posterior_params_se["e_c"], posterior_params_se["w_c"] = parameterisation_se.convert_secosw_sesinw_to_e_w(posterior_params_se["secosw_c"], posterior_params_se["sesinw_c"])


# use the MCMC samples and the stellar mass distribution to get a distribution for Mp sin(i)

mpsini_b_se = calculate_mpsini(mstar, posterior_params_se["P_b"], posterior_params_se["K_b"], posterior_params_se["e_b"], unit="M_earth")
mpsini_c_se = calculate_mpsini(mstar, posterior_params_se["P_c"], posterior_params_se["K_c"], posterior_params_se["e_c"], unit="M_earth")

# calculate the median and 1-sigma uncertainties
perc_b_se = np.percentile(mpsini_b_se, [16, 50, 84])
perc_c_se = np.percentile(mpsini_c_se, [16, 50, 84])
print("Planet b Mpsin(i):", perc_b_se[1], "+", perc_b_se[1] - perc_b_se[0], "-", perc_b_se[2] - perc_b_se[1])
print("Planet c Mpsin(i):", perc_c_se[1], "+", perc_c_se[1] - perc_c_se[0], "-", perc_c_se[2] - perc_c_se[1])

# Plot the mass posteriors for inspection
plt.hist(mpsini_b_se, bins=100, histtype="step")
plt.hist(mpsini_c_se, bins=100, histtype="step")
plt.show()